In [26]:
%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
import glob2
import pandas as pd
import re

In [2]:
# downloading data that are fit files
all_fit_files = glob2.glob(r'C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\**\*.fits',recursive=True)

## Creating Data Frame with fit files

In [144]:
TICID_list = []
sector_list = []
file_type = []
file_name = []

for i in all_fit_files:
    tuples = re.findall(r'-s00(\d\d)-0+(\d+)-\d+-(.+)\.fits', i)
    
    for l in tuples:
        sector_list += [l[0]]
        TICID_list += [l[1]]
        file_type += [l[2]]
        file_name += [i]

In [145]:
d = {'file_name':file_name,'TICID': TICID_list, 'SECTOR': sector_list, 'file_type':file_type}
df = pd.DataFrame(data =d)
df.head()

,file_name,TICID,SECTOR,file_type
0,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,a_fast-lc
1,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,a_fast-tp
2,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,s_lc
3,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,s_tp
4,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,27,a_fast-lc


In [150]:
dvt_files = list(set(all_fit_files) - set(df["file_name"])) #list of dvt files

### Some Files don't have PDCSAP flux
If the file does have pdcsap flux then column 'PDCSAP_FLUX' == 'TRUE'

In [169]:
df["PDCSAP_FLUX"] = ['FALSE']*len(df["file_name"])
for index in df.index:
    with fits.open(df.loc[index,'file_name'], mode="readonly") as hdulist:
        cols = hdulist[1].columns
        for i in cols.names:
            if i == 'PDCSAP_FLUX': #for files that do have PDCSAP_FLUX parameter
                df.loc[index,'PDCSAP_FLUX'] = 'TRUE'

In [173]:
df.head()

,file_name,TICID,SECTOR,file_type,PDCSAP_FLUX
0,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,a_fast-lc,TRUE
1,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,a_fast-tp,FALSE
2,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,s_lc,TRUE
3,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,24706112,27,s_tp,FALSE
4,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,27,a_fast-lc,TRUE


### Defining selected files of interest 
Creating a new data frame with selected TICID and merging with Gaia Data of those objects

In [194]:
# creating data frame of interested objects
selected = ['25133286', '764425749', '350152284', '364266095', '630952812', '270674067', '25575692'] # selected TICID
selected_data = df[df["TICID"].isin(selected)]
#selected_data.head()

# creating data fram for 'maybe' list
maybe_list = ['140579075', '764653535', '177118939', '675038317', '737123430', '737578301', '737578901', '765549141', '685352438', '270674067', '737578301', '25554524', '737648490']
maybe_data = df[df["TICID"].isin(maybe_list)]
maybe_data.head()

,file_name,TICID,SECTOR,file_type,PDCSAP_FLUX
12,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,a_fast-lc,TRUE
13,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,a_fast-tp,FALSE
14,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,s_lc,TRUE
15,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,s_tp,FALSE
24,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,177118939,27,a_fast-lc,TRUE


In [197]:
# Loading Gaia data
gaia_data = pd.read_csv('tess_2_gaia.txt', sep=",")
gaia_data.head()

,tic_id,gaia_dr2,ra,dec,name,simbad
0,25133286,4653893040002306432,62.297486,-71.294876,VW Hyi,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...
1,25575692,3099971986824957184,98.613965,-7.301006,NaN,NaN
2,270674067,4794480174438869632,87.439212,-49.365639,AR Pic,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...
3,350152284,4626488301092340864,53.102055,-78.564364,NaN,NaN
4,364266095,4681704263210424960,62.788548,-59.187835,NaN,NaN


In [210]:
gaia_data['tic_id'] = [str(x) for x in gaia_data['tic_id']]

In [223]:
# merging gaia data
selected = selected_data.merge(gaia_data, left_on='TICID', right_on='tic_id').drop(columns='tic_id')
maybe = maybe_data.merge(gaia_data, left_on='TICID', right_on='tic_id').drop(columns='tic_id')

In [227]:
selected.head()

,file_name,TICID,SECTOR,file_type,PDCSAP_FLUX,gaia_dr2,ra,dec,name,simbad
0,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,27,a_fast-lc,TRUE,4653893040002306432,62.297486,-71.294876,VW Hyi,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...
1,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,27,a_fast-tp,FALSE,4653893040002306432,62.297486,-71.294876,VW Hyi,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...
2,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,27,s_lc,TRUE,4653893040002306432,62.297486,-71.294876,VW Hyi,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...
3,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,27,s_tp,FALSE,4653893040002306432,62.297486,-71.294876,VW Hyi,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...
4,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,25133286,29,a_fast-lc,TRUE,4653893040002306432,62.297486,-71.294876,VW Hyi,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...


### Adding the rest of Gaia Data 

In [237]:
# query used on gaia archive for selected data frame
# SELECT TOP 500 gaia_source.source_id,gaia_source.bp_rp,gaia_source.bp_g,gaia_source.g_rp
# FROM gaiadr2.gaia_source 
# WHERE (gaiadr2.gaia_source.source_id IN (4653893040002306432, 4640227970191399424, 5206772708159305600,
#  4626488301092340864, 4681704263210424960, 4794480174438869632))

In [236]:
# loading gaia data for selected objects
sel_gaia = pd.read_csv('selected_gaiadata.csv', sep=",")
# merging with df
selected = selected.merge(sel_gaia, left_on='gaia_dr2', right_on='source_id').drop(columns='source_id')

# loading gaia data for maybe list
maybe_gaia = pd.read_csv('maybe_gaiadata.csv', sep=",")
# merging with df
maybe = maybe.merge(maybe_gaia, left_on='gaia_dr2', right_on='source_id').drop(columns='source_id')
maybe.head()

,file_name,TICID,SECTOR,file_type,PDCSAP_FLUX,gaia_dr2,ra,dec,name,simbad,bp_rp,bp_g,g_rp
0,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,a_fast-lc,TRUE,4627494526030330112,68.220983,-77.608424,EC 04350-7742,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...,0.900374,0.047614,0.85276
1,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,a_fast-tp,FALSE,4627494526030330112,68.220983,-77.608424,EC 04350-7742,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...,0.900374,0.047614,0.85276
2,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,s_lc,TRUE,4627494526030330112,68.220983,-77.608424,EC 04350-7742,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...,0.900374,0.047614,0.85276
3,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,27,s_tp,FALSE,4627494526030330112,68.220983,-77.608424,EC 04350-7742,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...,0.900374,0.047614,0.85276
4,C:\Users\fatim\CV_Research_Folder\TESS_DATA_1\...,140579075,28,a_fast-lc,TRUE,4627494526030330112,68.220983,-77.608424,EC 04350-7742,http://simbad.u-strasbg.fr/simbad/sim-id?Ident...,0.900374,0.047614,0.85276
